In [1]:
import numpy as np
import math

In [2]:
data = np.load('./data/Direction_task_with_dots_synchronised_min.npz')
X = data['EEG']
y = data['labels']

In [3]:
scoring2 = (lambda y, y_pred: np.sqrt(np.mean(np.square(np.arctan2(np.sin(y - y_pred), np.cos(y - y_pred))))))

In [4]:
cnn_pred = np.load('./runs/1631607045_Direction_task_dots_min_pytorch_patience_20_lr_1e-05/checkpoint/run1/CNN_pred.npz')

In [5]:
pred = cnn_pred['pred']

In [6]:
import math 

def split(ids, train, val, test):
    assert (train+val+test == 1)
    IDs = np.unique(ids)
    num_ids = len(IDs)

    # priority given to the test/val sets
    test_split = math.ceil(test * num_ids)
    val_split = math.ceil(val * num_ids)
    train_split = num_ids - val_split - test_split

    train = np.isin(ids, IDs[:train_split])
    val = np.isin(ids, IDs[train_split:train_split+val_split])
    test = np.isin(ids, IDs[train_split+val_split:])

    return train, val, test

In [7]:
trainX = X 
trainY = y
ids = trainY[:, 0]

In [8]:
train, val, test = split(ids, 0.7, 0.15, 0.15)
X_train, y_train = trainX[train], trainY[train]
X_val, y_val = trainX[val], trainY[val]
X_test, y_test = trainX[test], trainY[test]

In [9]:
labels = y_test[:,2]
labels

array([ 0.6388761 , -2.24920304, -0.01477896, ...,  2.88190077,
        2.88778389,  2.75649705])

In [10]:
len(pred)

4075

In [11]:
len(y)

29714

In [18]:
# note that min(labels) = -3.14 and max(labels) = 3.14
NUM_DIRS = 4

bins = [[] for i in range(4)]
strings = ["bin 0: pi/4 to -pi/4, i.e. horizontal right", 
"bin 1: pi/4 to 3/4 pi, i.e. vertical up",
"bin 2: 3/4 pi to 5/4 pi = -3/4 pi, i.e. horizontal left",
"bin 3: 5/4 pi = -3/4 pi to -pi/4 = 7/4 pi, i.e. vertical down"]
for i in range(len(labels)):
	label = labels[i]
	if label > - math.pi/ 4 and label<= math.pi / 4:
		bins[0].append(scoring2(label, pred[i][0]))
	elif label > math.pi/4 and label <= 0.75 * math.pi:
		bins[1].append(scoring2(label, pred[i][0]))
	elif label > 0.75 * math.pi or label <= -0.75 * math.pi:
		bins[2].append(scoring2(label, pred[i][0]))
	elif label > -0.75 * math.pi and label <= -math.pi/4:
		bins[3].append(scoring2(label, pred[i][0]))

In [19]:
sum([len(list) for list in bins])

4075

In [20]:
for list, string in zip(bins, strings):
	print(f"{string}: mean angle error {np.average(np.array(list)):.3f}, std: {np.std(np.array(list)):2f}, on {len(list)} samples")

bin 0: pi/4 to -pi/4, i.e. horizontal right: mean angle error 0.229, std: 0.220355, on 1215 samples
bin 1: pi/4 to 3/4 pi, i.e. vertical up: mean angle error 0.271, std: 0.329660, on 669 samples
bin 2: 3/4 pi to 5/4 pi = -3/4 pi, i.e. horizontal left: mean angle error 0.188, std: 0.205688, on 1358 samples
bin 3: 5/4 pi = -3/4 pi to -pi/4 = 7/4 pi, i.e. vertical down: mean angle error 0.284, std: 0.386571, on 833 samples
